# Data Acquisition

The data that we will be using in the project will come from files provided in the kaggle challenge, "Predict Future Sales: Final project for "How to win a data science competition" Coursera course." The challenge includes a training set file of daily sales data, a test set file of store-item pairs, and a set of files as supplemental information for shops, item categories, and individual items. Since test set contains only store-item pairs and no target variable, we will focus only with training file. We will use training file to train our models and also use the last month sales record as hold out set to compare models.

In [2]:
% pylab inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [3]:
df_train = pd.read_csv('sales_train_v2.csv')
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [16]:
df_train.date = pd.to_datetime(df_train.date, format = '%d.%m.%Y')
df_train.set_index('date', inplace=True)
df_train = df_train.sort_index()
print(df_train.head())
print(df_train.info())

            date_block_num  shop_id  item_id  item_price  item_cnt_day
date                                                                  
2013-01-01               0       18     5823      2500.0           1.0
2013-01-01               0       27     5573       849.0           1.0
2013-01-01               0        7     1006       399.0           1.0
2013-01-01               0       19    17707       899.0           1.0
2013-01-01               0       14    19548       149.0           1.0
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2935849 entries, 2013-01-01 to 2015-10-31
Data columns (total 5 columns):
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3)
memory usage: 134.4 MB
None


In [17]:
df_train.isnull().sum()

date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

There are no null values in the data set.

In [18]:
# Total number of shops in training set
len(df_train.shop_id.unique())

60

In [19]:
# Number of shops that have sales record in Sept 2015
setp_data = df_train.loc[df_train['date_block_num'] ==32]
setp_shops = setp_data.shop_id.unique()
len(setp_shops)

43

In [20]:
# Number of shops that have sales record in Oct 2015 (last month)
oct_data = df_train.loc[df_train['date_block_num'] ==33]
oct_shops = oct_data.shop_id.unique()
len(oct_shops)

44

There are total of 60 shops in the entire training data. However, there are only 43 and 44 shops remaining in the last two months of the data. Therefore, we will use only the overlapping shops between the last two months for our project.

In [21]:
# Number of overlapping shops between Sept and Oct of 2015
overlap_shops = np.intersect1d(setp_shops,oct_shops)
print(len(overlap_shops))

41


There are only 41 shops that overlapped in both months, we will extract only the 41 shops from the training data.

In [22]:
df_shops = df_train.loc[df_train['shop_id'].isin(overlap_shops)]

Next, we will look at the breadth of SKUs, which are the item IDs in this case.

In [23]:
# Total number of items in training set
len(df_train.item_id.unique())

21807

In [24]:
setp_items = setp_data.item_id.unique()
print('Number of different items in Sept 2015: %i' % len(setp_items))

oct_items = oct_data.item_id.unique()
print('Number of different items in Oct 2015: %i' % len(oct_items))

overlap_items = np.intersect1d(setp_items,oct_items)
print('Number of overlapping items in the last two months of data: %i' % len(overlap_items))

Number of different items in Sept 2015: 5085
Number of different items in Oct 2015: 5413
Number of overlapping items in the last two months of data: 4041


There are way fewer items sold in both of the last two months compared to the entire SKUs. However, that is still way more store-item pairs than we are able to handle with bottom up method in calculating monthly sales of items in each shop. Therefore, we will limit the number of time series that we forecast to the top 50 popular items sold overall.

In [25]:
# Obtain total number sold per item from data set
df_item = df_shops.loc[df_shops['item_id'].isin(overlap_items)]
df_item = df_item.groupby(['item_id'])['item_cnt_day'].sum().sort_index()
df_item = pd.DataFrame(df_item)
df_item = df_item.rename(index=str, columns={"item_cnt_day": "item_cnt"})
df_item = df_item.reset_index().astype('int64')

# Sort items by numbers sold
top_selling_items = df_item.sort_values(by=['item_cnt'], ascending=False)

# Obtain top 50 most popular items and number sold
top_selling_50 = top_selling_items.head(50)

top_selling_50 = top_selling_50[['item_id', 'item_cnt']].reset_index()
top_selling_50 = top_selling_50[['item_id', 'item_cnt']]
print(top_selling_50.head())

   item_id  item_cnt
0    20949    154061
1     2808     13954
2     3732     13580
3    17717     13552
4     5822     12267


In [26]:
# Filter data set with top selling items
df = df_shops.loc[df_shops['item_id'].isin(top_selling_50.item_id)]

In [27]:
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
date,,,,,
2013-01-01,0,18,5823,2500.0,1.0
2013-01-01,0,46,1830,599.0,3.0
2013-01-01,0,14,1830,599.0,2.0
2013-01-01,0,14,1905,249.0,1.0
2013-01-01,0,14,2808,999.0,2.0


In [28]:
# Calculate weights: obtain total sales of each item in the last month in training set (2015-9), obtain last month sales of item in each shop, 
#    divide each last month sales by total item sales
df_hold = df.loc[df['date_block_num'] == 33]
df_hold = df_hold.groupby(['shop_id','item_id'])['item_cnt_day'].sum().sort_index()
df_hold = df_hold.reset_index()
df_hold = df_hold.rename(index=str, columns={"item_cnt_day": "item_cnt"})
print(df_hold.head())
print(len(df_hold))

   shop_id  item_id  item_cnt
0        2     2269       4.0
1        2     2445       1.0
2        2     2808       1.0
3        2     3341       4.0
4        2     3343       2.0
1025


In [29]:
len(df_hold.shop_id.unique())

40

After filtering the data from number of shops available in the last two months and the top selling items, there are only total of 40 shops that carried those items. In the end, there are total of 1025 different store-item pairs for calculation. Both filted data set and hold out set will be pickled for later use.

In [30]:
import pickle
df.to_pickle('df.pkl')
df_hold.to_pickle('df_hold.pkl')